In [1]:
import pandas as pd
from sqlalchemy import create_engine
from secret import username, password

### Extract CSVs into DataFrames

In [2]:
actors_file = "etl_project_data/IMDb names.csv"
actors_df = pd.read_csv(actors_file)


In [3]:
actors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175715 entries, 0 to 175714
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   imdb_name_id           175715 non-null  object 
 1   name                   175715 non-null  object 
 2   birth_name             175715 non-null  object 
 3   height                 30080 non-null   float64
 4   bio                    122832 non-null  object 
 5   birth_details          75229 non-null   object 
 6   birth_year             75111 non-null   float64
 7   date_of_birth          75102 non-null   object 
 8   place_of_birth         71122 non-null   object 
 9   death_details          26862 non-null   object 
 10  death_year             26838 non-null   float64
 11  date_of_death          26855 non-null   object 
 12  place_of_death         25036 non-null   object 
 13  reason_of_death        11972 non-null   object 
 14  spouses                175715 non-nu

### Transform premise DataFrame

In [4]:
# Create a filtered dataframe from specific columns
#actors_cols = ["imdb_name_id", "name", "birth_name", "height", "date_of_birth", "date_of_death", "primary_profession", "known_for_titles"]

actors_cols = ["name", "birth_name", "height", "date_of_birth", "date_of_death","primary_profession"]

actors_transformed= actors_df[actors_cols].copy()

In [5]:
# Comma separated value in the column transform into multiple columns

actorscleancsv = pd.concat([actors_transformed, actors_transformed["primary_profession"].str.split(',', expand=True)], axis=1)


In [6]:
# Rename the column headers
actorscleancsv = actorscleancsv.rename(columns={"date_of_birth": "birth_date", 
                                                0  : "Profession1",
                                                1 : "Profession2",
                                                2: "Profession3",
                                                          "date_of_death": "death_date"})


actorscleancsv.drop('primary_profession', axis=1, inplace=True)


In [7]:
# Retrieve only rows where profession is actor or actress

filter1 = actorscleancsv['Profession1'].isin(["actor", "actress"]) 
filter2 = actorscleancsv['Profession2'].isin(["actor", "actress"])  
filter3 = actorscleancsv['Profession3'].isin(["actor", "actress"])  
  
# displaying data with both filter applied and mandatory  
actors_filtered = actorscleancsv.loc[(filter1) | (filter2) | (filter3)] 

In [8]:
actorscleancsv.drop(['Profession1', 'Profession2', 'Profession3'], axis=1, inplace=True)


In [9]:
actorscleancsv['actor_id'] = range(1, len(actorscleancsv)+1)
actorscleancsv.head(25)


,name,birth_name,height,birth_date,death_date,actor_id
0,Fred Astaire,Frederic Austerlitz Jr.,177.0,1899-05-10,1987-06-22,1
1,Lauren Bacall,Betty Joan Perske,174.0,1924-09-16,2014-08-12,2
2,Brigitte Bardot,Brigitte Bardot,166.0,1934-09-28,NaN,3
3,John Belushi,John Adam Belushi,173.0,1949-01-24,1982-03-05,4
4,Ingmar Bergman,Ernst Ingmar Bergman,179.0,1918-07-14,2007-07-30,5
5,Ingrid Bergman,Ingrid Bergman,175.0,1915-08-29,1982-08-29,6
6,Humphrey Bogart,Humphrey DeForest Bogart,173.0,1899-12-25,1957-01-14,7
7,Marlon Brando,Marlon Brando Jr.,175.0,1924-04-03,2004-07-01,8
8,Richard Burton,Richard Walter Jenkins,175.0,1925-11-10,1984-08-05,9
9,James Cagney,James Francis Cagney,169.0,1899-07-17,1986-03-30,10


### Create database connection

In [10]:
connection_string = f'{username}:{password}@localhost:5432/ETL_project_movies'
engine = create_engine(f'postgresql://{connection_string}')

In [11]:
# Confirm tables
engine.table_names()

['movie',
 'movie_actor',
 'actor',
 'lang_movie',
 'lang',
 'movie_genre',
 'genre',
 'movie_country',
 'country',
 'movie_drt',
 'director',
 'rating']

### Load DataFrames into database

In [12]:
actorscleancsv.to_sql(name='actor', con=engine, if_exists='append', index=False)